In [1]:
import os
import sys
import numpy as np
import pandas as pd
import pickle

from feature_engine.imputation import MeanMedianImputer, ArbitraryNumberImputer
from feature_engine.outliers import Winsorizer
from feature_engine.wrappers import SklearnTransformerWrapper

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler , MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap
from sklearn.pipeline import Pipeline 

s_path = os.path.dirname(os.path.realpath(__file__))
sys.path.append(s_path)

pd.set_option('display.max_columns',100)
pd.set_option('precision', 3)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import tm_teoriaMvto_base_prep as base_prep
import tm_teoriaMvto_label as tm_label
import tm_teoriaMvto_train as tm_train
import tm_teoriaMvto_ft_eng as ft_eng
# import tm_teoriaMvto_ft_sel as ft_sel

# models to test
from sklearn.svm import SVC
import xgboost as xgb


In C:\Users\Desktop\Anaconda3\envs\ngym36\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Desktop\Anaconda3\envs\ngym36\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Desktop\Anaconda3\envs\ngym36\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\Desktop\Anaconda3\envs\ngym36\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Desktop\Anaconda3\envs\n

In [2]:
path_files = os.path.join(s_path, 'historical_files')

# base prep control
EXPORT_X = False
EXPORT_Y = False

# params labeling
s_prefix = 'ft_'
s_lbl_type = 'c_binary'
f_th = 0.2
b_dist = True
b_percent = True
b_custom = True

# params loading
s_regime = 'mi_up'



In [3]:
if EXPORT_X:
  df_data = base_prep.import_sampling(path_files= os.path.join(path_files, 'FeaturesFiles'), prefix_files= s_prefix)

  # create new features before extracting samples - using full data needed by lag features
  df_data = ft_eng.BasicFeatures().transform(X=df_data)

  df_X = base_prep.ft_export(df_data, path_files= os.path.join(path_files, 'TrainFiles'), prefix= s_prefix)
  y_sc = base_prep.y_export(df_data, path_files= os.path.join(path_files, 'TrainFiles'), prefix= s_prefix)



In [4]:
if EXPORT_Y:
  df_label = None 
  if EXPORT_X: df_label = y_sc

  # specify df_data = None (default) to load pickle from s_path + 'y_prep_data.pkl'
  label = tm_label.Labeling(df_data = df_label, label_type= s_lbl_type, b_dist_to_high= b_dist, 
                            s_path = os.path.join(path_files, 'TrainFiles'), prefix_files = s_prefix, th_label_y1=f_th, 
                            b_percent_freq=b_percent, b_custom_dir=b_custom)

  df_y = label.apply_label(s_model_return = s_regime)  # it will export 4 pickle files to label_obj.s_path
  y_config = label.s_name
else:
  # choose y_config
  y_config = 'prefix_'+ s_prefix +'_type_'+ s_lbl_type +'_th_'+ str(f_th) +'_dist_'+ str(b_dist) +'_percent_'+str(b_percent)+'_custom_'+str(b_custom)

y_outfile = os.path.join(os.path.join(path_files, 'TrainFiles'), 'y_' + y_config +'.pkl')

if os.path.exists(y_outfile):
  print('y outfile ready for configuration: {}'.format(y_config))
else:
  print('WARNING: y outfile not found for configuration. Export y before continuing {}'.format(y_config))



y outfile ready for configuration: prefix_ft__type_c_binary_th_0.2_dist_True_percent_True_custom_True


In [5]:
l_col_08 = ['escora_bid_2.5_0.8', 'escora_ask_2.5_0.8', 'escora_bid_3.5_0.8', 
            'escora_ask_3.5_0.8', 'escora_bid_4.5_0.8', 'escora_ask_4.5_0.8', ]

l_col_12 = ['escora_bid_2.5_1.2', 'escora_ask_2.5_1.2', 'escora_bid_3.5_1.2',
            'escora_ask_3.5_1.2', 'escora_ask_4.5_1.2', 'escora_bid_4.5_1.2', ]

l_col_2 = [ 'escora_bid_2.5_2', 'escora_ask_2.5_2', 'escora_bid_3.5_2',
            'escora_ask_3.5_2', 'escora_bid_4.5_2', 'escora_ask_4.5_2', ]

l_col_log = ['agg_net_d', 'aggbig_net_d', 'vol_trd', 'vol_big', 'big_v', 'vol_trd_aux', 
   'vol_big_aux', 'big_v_aux', 'loc_agg_net_d', 'big_c', 'big_c_aux',
   'loc_aggbig_net_d', 'agg_net_m', 'agg_net_m_aux', 'abagg', 'abagg_aux',
   'aggbig_net_m', 'aggbig_net_m_aux', 'loc_agg_net_m', 'loc_aggbig_net_m',
   'loc_agg_net_m_aux', 'loc_aggbig_net_m_aux', 'loc_aggbig_c_m', 'loc_aggbig_v_m', 
   'loc_aggbig_c_m_aux', 'loc_aggbig_v_m_aux', 'abs_v', 'abs_c', 'aggpior_v', 'aggpior_v_aux', 
   'aggpior_c', 'aggpior_c_aux', 'agg_net_10', 'agg_net_40', 'agg_net_80', 'loc_agg_net_10',
   'aggbig_net_10', 'aggpior_DIF', 'aggpior_DIF_30', 'abs_DIF', 'abs_DIF_30',
   'abagg_10', 'aggpior_aux_DIF', ]

l_side_drop = ['big_c','big_v','aggpior_c','aggpior_v','loc_aggbig_c_m','loc_aggbig_v_m','pagg_c_best',
  'pagg_c_best_0.5','pagg_c_best_0.7','pagg_c_best_0.9','pagg_v_best','pagg_v_best_0.5','pagg_v_best_0.7',
  'pagg_v_best_0.9','abs_c','abs_v','int_c','int_c_0.6','int_c_0.7','int_c_0.8','int_c_0.9','int_dif_c',
  'int_v','int_v_0.6','int_v_0.7','int_v_0.8','int_v_0.9','int_dif_v','imp_c','imp_c_0.6','imp_c_0.7',
  'imp_c_0.8','imp_c_0.9','imp_v','imp_v_0.6','imp_v_0.7','imp_v_0.8','imp_v_0.9','escora_bid_2.5_1.2',
  'escora_bid_2.5_2','escora_ask_2.5_1.2','escora_ask_2.5_2','escora_bid_3.5_1.2','escora_bid_3.5_2',
  'escora_ask_3.5_1.2','escora_ask_3.5_2','escora_bid_4.5_0.8','escora_bid_4.5_2','escora_ask_4.5_0.8',
  'escora_ask_4.5_2','movesc_bid_2.5','movesc_ask_2.5','movesc_bid_2.5_0.5','movesc_ask_2.5_0.5',
  'movesc_bid_2.5_0.7','movesc_ask_2.5_0.7','movesc_bid_3.5','movesc_ask_3.5','movesc_bid_3.5_0.7',
  'movesc_ask_3.5_0.7','movesc_bid_3.5_0.9','movesc_ask_3.5_0.9','movesc_bid_4.5','movesc_ask_4.5',
  'movesc_bid_4.5_0.5','movesc_ask_4.5_0.5','movesc_bid_4.5_0.7','movesc_ask_4.5_0.7','depth_bid7','depth_ask7'
  ]

l_side_drop_aux = ['big_c_aux','big_v_aux','aggpior_c_aux','aggpior_v_aux','loc_aggbig_c_m_aux',
  'loc_aggbig_v_m_aux','pagg_c_best_aux','pagg_c_best_0.5_aux','pagg_c_best_0.7_aux',
  'pagg_c_best_0.9_aux','pagg_v_best_aux','pagg_v_best_0.5_aux', 'pagg_v_best_0.7_aux',
  'pagg_v_best_0.9_aux'
  ]

l_ft_aux = [
  'vol_trd_aux', 'n_trd_aux','vol_big_aux','n_big_aux','vol_big_ratio_aux','big_c_aux','big_v_aux',
  'aggpior_c_aux','aggpior_v_aux','aggimb_aux','aggimb_big_aux','n_aggimb_aux','agg_net_m_aux',
  'aggbig_net_m_aux','loc_aggbig_c_m_aux','loc_aggbig_v_m_aux','loc_agg_net_m_aux','loc_aggbig_net_m_aux',
  'loc_agg_imb_m_aux','loc_aggbig_imb_m_aux','pagg_c_best_aux','pagg_c_best_0.5_aux','pagg_c_best_0.7_aux',
  'pagg_c_best_0.9_aux','pagg_v_best_aux','pagg_v_best_0.5_aux','pagg_v_best_0.7_aux','pagg_v_best_0.9_aux',
  'abagg_aux','n_p_aux','aggpior_aux_DIF','pagg_aux_DIF'
  ]


In [6]:
test_size = 0.2

if not EXPORT_X:
  df_X = tm_train.load_models('X_samples_'+s_prefix, os.path.join(path_files, 'TrainFiles'))  # search for path_files/s_regime.pkl

X = df_X.loc[df_X['model'] == s_regime].drop(columns='model')

if not EXPORT_Y:
  df_y = tm_train.load_models('y_' + y_config, os.path.join(path_files, 'TrainFiles'))
  y = df_y.loc[df_y['model'] == s_regime].drop(columns='model')
else:
  y = df_y.drop(columns='model')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, shuffle = False)

print('load_split: X, y data load and split complete!')



load_split: X, y data load and split complete!


In [7]:
# DEBUG: prop_features contaning np.NaN = (pd.isna(X_train).sum()>0).sort_values(ascending=False).head(50)
X_train = ft_eng.PropImputer(0.8, l_col_08).transform(X_train)
X_train = ft_eng.PropImputer(1.2, l_col_12).transform(X_train)
X_train = ft_eng.PropImputer(2, l_col_2).transform(X_train)

median_inputer = MeanMedianImputer(variables=['PA_down',])
X_train = median_inputer.fit_transform(X_train)

nan_imputer = ArbitraryNumberImputer(0.0, variables=['ohlc_10','ohlc_50'])
X_train = nan_imputer.fit_transform(X_train)

X_train = ft_eng.DifAll().transform(X_train)
X_train = ft_eng.LogVolume(l_col_log).transform(X_train)

if s_regime[:2] == 'mw':
  # for now, removing all side columns
  l_cols_drop = l_side_drop + l_ft_aux
else:
  l_cols_drop = l_side_drop + l_side_drop_aux

X_train.drop(columns=l_cols_drop, inplace=True)


In [8]:
X_test = ft_eng.PropImputer(0.8, l_col_08).transform(X_test)
X_test = ft_eng.PropImputer(1.2, l_col_12).transform(X_test)
X_test = ft_eng.PropImputer(2, l_col_2).transform(X_test)

X_test = median_inputer.transform(X_test)

X_test = nan_imputer.transform(X_test)

X_test = ft_eng.DifAll().transform(X_test)
X_test = ft_eng.LogVolume(l_col_log).transform(X_test)

if s_regime[:2] == 'mw':
  # for now, removing all side columns
  l_cols_drop = l_side_drop + l_ft_aux
else:
  l_cols_drop = l_side_drop + l_side_drop_aux

X_test.drop(columns=l_cols_drop, inplace=True)

train_obj = tm_train.ModelTraining()


In [9]:

pipe_preproc = Pipeline([
                ('scaler', SklearnTransformerWrapper(transformer=RobustScaler(quantile_range=(0.15, 0.85)))),
                ('outliers', Winsorizer(capping_method='quantiles', fold = 0.001, tail = 'both')),
                ('minmax', SklearnTransformerWrapper(transformer=MinMaxScaler(feature_range=(-1, 1)))),
                ('pca', PCA(n_components=30, svd_solver='auto')), 
                # ('isomap', Isomap(n_components=13, n_neighbors=50, n_jobs=-1)),   #  expensive
                ])

X_pca = pipe_preproc.fit_transform(X_train)
X_test_pca = pipe_preproc.transform(X_test)



In [10]:
clf_svm = SVC(probability=True, cache_size=1000, verbose=0
              ,class_weight= 'balanced'
              # ,C=1, gamma=0.001
              )

clf_svm.fit(X_pca, y_train) 
l_results = tm_train.report_results(X_pca, X_test_pca, y_train, y_test, fitted_model=clf_svm, ready_probs = False)

# forest_minimize 

train_obj.optimized_training(X_pca, y_train, 'svm', clf_svm, pipe = False)



C:\Users\Desktop\Anaconda3\envs\ngym36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
train_auc: 0.77, test_auc: 0.55 (overfit: 0.22)
train_f1:  0.54, test_f1:  0.27 (overfit: 0.26)
train_f*:0.65, test_f*:0.37 (overfit: 0.28)
--------
cm_train: 
[[6329  702]
 [3138 2220]]
cm_test: 
[[1512  268]
 [1068  250]]
threshold: 0.5
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 129.3601
Function value obtained: 0.4295
Current minimum: 0.4295
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 127.1825
Function value obtained: 0.4294
Current minimum: 0.4294
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation do

In [11]:

pipe_xgb = Pipeline([  
                    # TODO: feature_selection()
                    ('xgb', xgb.XGBClassifier(
                          max_depth= 4,
                          subsample= 0.4102805752324993,
                          colsample_bytree= 0.18592295315692304,
                          min_child_weight= 50,
                          learning_rate= 0.006666055995539396,
                          n_estimators= 418,
                          gamma= 1,
                          reg_alpha= 9.436346363300686,
                          scale_pos_weight= 7.107995018980205
                      ))
                    ])


# training and test scores 
# pipe_xgb.fit(X_train, y_train)
# l_results = tm_train.report_results(X_train, X_test, y_train, y_test, fitted_model=pipe_xgb, ready_probs = False)
# f_imp = tm_train.ft_importance_plot(pipe_xgb[1], pipe_xgb[0].transform(X_train), plot_result=True)



In [12]:

d_params = {
            # 'svc__class_weight': ['balanced', None],
            'svc__C': [.1, .5, 1, 10, 100], 
            'svc__gamma': [0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1, 0.3, 0.6]
            }

# l_results, clf_cv = tm_train.rscv_clf(pipe_svm, d_params, X_train, y_train, X_test, y_test)


In [13]:
train_obj.result_opt

          fun: 0.42940571053347776
    func_vals: array([0.42950246, 0.42941107, 0.4303829 , 0.42941095, 0.43855993,
       0.42940571])
       models: []
 random_state: RandomState(MT19937) at 0x25A3D82CBA0
        space: Space([Real(low=1e-06, high=100.0, prior='log-uniform', transform='identity'),
       Real(low=1e-07, high=10.0, prior='log-uniform', transform='identity')])
        specs: {'args': {'model_queue_size': None, 'n_jobs': -1, 'kappa': 2.0, 'xi': 0.01, 'n_restarts_optimizer': 5, 'n_points': 10000, 'callback': <skopt.callbacks.DeltaYStopper object at 0x0000025A3D6A7A20>, 'verbose': 1, 'random_state': None, 'y0': None, 'x0': None, 'acq_optimizer': 'sampling', 'acq_func': 'EI', 'initial_point_generator': 'random', 'n_initial_points': 10, 'n_random_starts': None, 'n_calls': 30, 'base_estimator': 'ET', 'dimensions': [Real(low=1e-06, high=100.0, prior='log-uniform', transform='identity'), Real(low=1e-07, high=10.0, prior='log-uniform', transform='identity')], 'func': <function

In [14]:
clf_svm = SVC(probability=True, cache_size=1000, verbose=0
              ,class_weight= 'balanced'
              ,C=1.26, gamma=0.0002
              )

clf_svm.fit(X_pca, y_train) 
l_results = tm_train.report_results(X_pca, X_test_pca, y_train, y_test, fitted_model=clf_svm, ready_probs = False)


C:\Users\Desktop\Anaconda3\envs\ngym36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
train_auc: 0.60, test_auc: 0.56 (overfit: 0.04)
train_f1:  0.60, test_f1:  0.56 (overfit: 0.04)
train_f*:0.52, test_f*:0.49 (overfit: 0.03)
--------
cm_train: 
[[2551 4480]
 [1169 4189]]
cm_test: 
[[ 654 1126]
 [ 369  949]]
threshold: 0.4


In [15]:
X_train

93000 870             -4.134         -3.045   
           63011  09:29:48.428000 1137             2.904          3.258   
           67015  09:30:47.568000 1263            -2.550          0.000   
...                                                  ...            ...   
2021-01-21 935156 14:51:18.956000 21691           -4.981         -3.932   
           936015 14:51:42.285000 21708           -5.239          3.932   
           943637 14:55:13.748000 21898           -6.437         -3.932   
           948326 14:56:20.909000 22061            5.968         -4.111   
           961349 15:03:55.529000 22282           -6.573         -3.258   

                                          int_DIF  int_DIF_10  int_DIF_50  \
date_trd   update time            trd_id                                    
2020-06-01 49460  09:21:41.989000 764      -0.007       0.000      -0.045   
           52825  09:23:20.039000 860       0.068       0.000       0.004   
           53165  09:23:31.093000 870       0.071       0.003       0.006   
           63011  09:29:48.428000 1137     -0.044      -0.010      -0.094   
           67015  09:30:47.568000 1263     -0.411       0.000      -0.150   
...                                           ...         ...         ...   
2021-01-21 935156 14:51:18.956000 21691     0.246       0.076      -0.177   
           936015 14:51:42.285000 21708     0.108      -0.026       0.075   
           943637 14:55:13.748000 21898    -0.119      -0.038       0.052   
           948326 14:56:20.909000 22061    -0.486      -0.237      -0.261   
           961349 15:03:55.529000 22282     0.114       0.047       0.135   

                                          imp_FCAST_10  imp_FCAST_40  \
date_trd   update time            trd_id                               
2020-06-01 49460  09:21:41.989000 764            0.000        -1.575   
           52825  09:23:20.039000 860           -0.972        -6.028   
           53165  09:23:31.093000 870           -1.363        -2.727   
           63011  09:29:48.428000 1137          -0.784        -0.627   
           67015  09:30:47.568000 1263          -0.077        -3.063   
...                                                ...           ...   
2021-01-21 935156 14:51:18.956000 21691         -0.270        -0.406   
           936015 14:51:42.285000 21708         -0.273        -0.121   
           943637 14:55:13.748000 21898         -0.606        -2.006   
           948326 14:56:20.909000 22061         -0.756        -4.987   
           961349 15:03:55.529000 22282          0.412        -3.299   

                                          aggpior_DIF  aggpior_DIF_30  \
date_trd   update time            trd_id                                
2020-06-01 49460  09:21:41.989000 764          -4.263           0.000   
           52825  09:23:20.039000 860          -3.714          -3.045   
           53165  09:23:31.093000 870          -4.111          -3.714   
           63011  09:29:48.428000 1137          0.000           0.000   
           67015  09:30:47.568000 1263         -4.454          -4.454   
...                                               ...             ...   
2021-01-21 935156 14:51:18.956000 21691         0.000           3.045   
           936015 14:51:42.285000 21708         0.000           3.045   
           943637 14:55:13.748000 21898         4.331           5.916   
           948326 14:56:20.909000 22061        -3.714           4.394   
           961349 15:03:55.529000 22282         0.000           0.000   

                                          abs_DIF  abs_DIF_30  pagg_DIF  \
date_trd   update time            trd_id                                  
2020-06-01 49460  09:21:41.989000 764       0.000      -4.984    -1.000   
           52825  09:23:20.039000 860       0.000       0.000     0.000   
           53165  09:23:31.093000 870       0.000       0.000     0.000   
           63011  09:29:48.428000 1137      0.000       0.000     0.000   
           67

In [16]:
X_train.dtypes 

vol_trd               float64
n_trd                   int64
vol_big               float64
n_big                   int64
vol_big_ratio         float64
                       ...   
escora_3.5_1.2_DIF    float64
escora_4.5_0.8_DIF    float64
movesc_2.5_0.7_DIF    float64
movesc_3.5_0.7_DIF    float64
movesc_4.5_0.7_DIF    float64
Length: 116, dtype: object